## Header for selecting leftout compound directory to work in

In [17]:
loo_pert_id_idx = 1


loo_pert_id_idx =

     1



In [18]:
unique_kabx_cmpds_tbl = rtable('/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/loocv_pcls/kabx_pert_ids_tbl_for_loocv.txt');

size(unique_kabx_cmpds_tbl)
headt(unique_kabx_cmpds_tbl)

Reading /idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/loocv_pcls/kabx_pert_ids_tbl_for_loocv.txt


ans =

   434     2


ans =

  2x3 table

    idx          field                value      
    ___    _________________    _________________

     1     {'kabx_cmpd_idx'}    {[            1]}
     2     {'kabx_cmpd'    }    {'BRD-A02179977'}



In [19]:
this_loocv_cmpd = unique_kabx_cmpds_tbl.kabx_cmpd(unique_kabx_cmpds_tbl.kabx_cmpd_idx == loo_pert_id_idx)

this_loocv_cmpd_string = strjoin(this_loocv_cmpd)


this_loocv_cmpd =

  1x1 cell array

    {'BRD-A02179977'}


this_loocv_cmpd_string =

    'BRD-A02179977'



## End Header

In [ ]:
% Added by AB on 06/12/2024
% Ensure parallel pool is cleaned up at the end of the script or on error
cleanupObj = onCleanup(@() delete(gcp('nocreate')));

In [2]:
datadir = '/idi/cgtb/morzech/idmp/combine_screens_456';
wkdir = ['/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript_final/loocv_pcls/', 'leave_out_cmpd_', this_loocv_cmpd_string]
%wkdir = fullfile(datadir,'pcls_remove_gsk_no_broad_sar_pearson');
outdir = fullfile(wkdir,'pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den_fast')

mk_cd_dir(outdir,false);
mk_cd_dir(wkdir, true);
imatlab_export_fig('print-png')


wkdir =

    '/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript'


outdir =

    '/idi/cgtb/abond/combine_screens_456/pcls_for_manuscript/pcls_spectral_clustering_thrsh_rank_le20_k_med_gap_den'



In [63]:
p = parpool(feature('numcores'),'IdleTimeout',120);

Error using parpool (line 145)
Found an interactive session. You cannot have multiple interactive sessions open simultaneously. To terminate the existing session, use <a href="matlab:delete(gcp('nocreate'))">delete(gcp('nocreate'))</a>.


In [3]:
g = glob(fullfile(wkdir,'grzs_for_pcls_screens456_pearson_corr_n*.gctx'));
c_all = parse_gctx(g{1});
n = numel(c_all.rid)

% g = glob(fullfile(wkdir,'grzs_corr_rank_n*.gctx'));
% cr_all = parse_gctx(g{1});

Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/grzs_for_pcls_screens456_corr_n13826x13826.gctx [13826x13826]
Done [9.32 s].

n =

       13826



In [4]:
[~,col_meta] = gct2meta(c_all);

gct2meta> Creating row and column metadata tables



In [5]:
headt(col_meta)


ans =

  33x3 table

    idx               field                                    value                      
    ___    ___________________________    ________________________________________________

     1     {'cid'                    }    {'screen6:BRD-A00069066-001-01-9:0.781250uM'   }
     2     {'broad_id'               }    {'BRD-A00069066-001-01-9'                      }
     3     {'canonical_smiles'       }    {'-666'                                        }
     4     {'cid_org'                }    {'-666'                                        }
     5     {'dose_series_id'         }    {'homebrew'                                    }
     6     {'n_replicates'           }    {[                                        -666]}
     7     {'pert_dose'              }    {[                                      0.7812]}
     8     {'pert_dose_act'          }    {[                                      0.7812]}
     9     {'pert_dose_unit'         }    {'uM'                    

In [6]:
col_meta.x_dose_series_id = any2str(col_meta.x_dose_series_id);

In [7]:
pcls = parse_gmt(fullfile(wkdir,'new_pcls.gmt'));

## Plot eigenvalues

In [10]:
thrsh_factor = 1;
thrsh_rank = 20;
%k_type = 'k_num_zero';
k_type = 'k_med_gap_den';
save_fig = true;
save_out = false;


out_lap = {};
for ii = 1:numel(pcls)
    loop_progress(ii,numel(pcls),10)
    
    % Set MoA class
    moa_class = pcls(ii).head;
    out_lap(ii).moa_class = moa_class;
    
    % Load gcts
    g = glob(fullfile(wkdir,'verify_pcls','gcts',[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_n*.gct']));
    c = parse_gctx(g{1});
    g = glob(fullfile(wkdir,'verify_pcls','gcts',[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_rank_n*.gct']));
    cr = parse_gctx(g{1});
    
    % Set threshold
    thrsh = thrsh_rank*thrsh_factor;
    out_lap(ii).thrsh_factor = thrsh_factor;
    out_lap(ii).thrsh = thrsh;

    % Create adjacency matrix
    adj_mat = double(cr.mat<=thrsh);

    % Normalized Laplacian
    [L,out_lap(ii).k,en,den,out_lap(ii).k_gap_den,out_lap(ii).k_num_zero] = create_laplacian_edit_ab(adj_mat,'symmetric',save_fig,k_type);
    
    moa_class_str = strrep(strrep(moa_class,' ','_'),'/','-');
    saveas_png(gcf,outdir,[moa_class_str,'_eigenvalues.png'])
    close(gcf)
end
out_lap = struct2table(out_lap);
wtable(out_lap,fullfile(outdir,'k_values.txt'))
disp('All done')

1/111
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2016_series_(topoisomerase_I)_corr_n64x64.gct [64x64]
class:single
read:64/64
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2016_series_(topoisomerase_I)_corr_rank_n64x64.gct [64x64]
class:single
read:64/64
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2626_series_corr_n112x112.gct [112x112]
class:single
read:112/112
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2626_series_corr_rank_n112x112.gct [112x112]
class:single
read:112/112
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/5F-THF-antidote_to_folate_inhibitors_corr_n20x20.gct [20x20]
class:single
read:20/20
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/5F-THF-antidote_to_folate_inhibitors_corr_rank_n20x20.gct [20x20]
class:singl

## Run Spectral Clustering

In [64]:
thrsh_rank = 20;
thrsh_factor = 1;
%k_type = 'k_num_zero';
k_type = 'k_med_gap_den';
save_fig = true;
save_out = true;


parfor ii = 1:numel(pcls)
    loop_progress(ii,numel(pcls),1)
    
    % Set MoA class
    moa_class = pcls(ii).head;
    disp(moa_class)
    [strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_n*.gct']
    % Load gcts
    g = glob(fullfile(wkdir,'verify_pcls','gcts',[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_n*.gct']))
    c = parse_gctx(g{1});
    g = glob(fullfile(wkdir,'verify_pcls','gcts',[strrep(strrep(moa_class,' ','_'),'/','-'),'_corr_rank_n*.gct']));
    cr = parse_gctx(g{1});
    
    % Set threshold
    thrsh = thrsh_rank*thrsh_factor;
    
    % Run spectral clustering
    try
        [spec(ii).tmp_out_nonsingle,spec(ii).tmp_out,spec(ii).tmp_gmt] = spectral_clust_for_pcls_fast(c,cr,moa_class,thrsh,k_type,outdir,save_out,save_fig);
        close all
    catch ME
        disp('Error from spectral clustering')
        disp(ME)
    end
end
disp('All done')

1/111
2016 series (topoisomerase I)

ans =

    '2016_series_(topoisomerase_I)_corr_n*.gct'

Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2016_series_(topoisomerase_I)_corr_n64x64.gct [64x64]
class:single
read:64/64
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2016_series_(topoisomerase_I)_corr_rank_n64x64.gct [64x64]
class:single
read:64/64
Done.
2/111
2626 series

ans =

    '2626_series_corr_n*.gct'

Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2626_series_corr_n112x112.gct [112x112]
class:single
read:112/112
Done.
Reading /idi/cgtb/morzech/idmp/combine_screens_456/pcls_remove_gsk/verify_pcls/gcts/2626_series_corr_rank_n112x112.gct [112x112]
class:single
read:112/112
Done.
All done


In [10]:
spec_bak = spec;

## Combine tables

In [51]:
spec = {};
out = {};
out_nonsingle = {};
gmt = {};

num_moas_represented = 0;

for ii = 1:numel(pcls)
    loop_progress(ii,numel(pcls),10)
    
    moa_class = pcls(ii).head;
    moa_class_str = strrep(strrep(moa_class,' ','_'),'/','-');
    
    out_path = fullfile(outdir,[moa_class_str,'_spectral_clust.txt']);
    out_nonsingle_path = fullfile(outdir,[moa_class_str,'_spectral_clust_nonsingle.txt']);
    gmt_path = fullfile(outdir,[moa_class_str,'_spectral_clust.gmt']);
    
    if exist(gmt_path, 'file') == 2
        spec(ii).tmp_out_nonsingle = rtable(out_nonsingle_path);
        spec(ii).tmp_out = rtable(out_path);
        spec(ii).tmp_gmt = parse_gmt(gmt_path);
        
        fields = unique([spec(ii).tmp_out_nonsingle.Properties.VariableNames,spec(ii).tmp_out.Properties.VariableNames]);
    
        for jj = 1:numel(fields)
            try
                spec(ii).tmp_out_nonsingle.(fields{jj}) = any2str(spec(ii).tmp_out_nonsingle.(fields{jj}));
            catch ME
                true;
            end

            try
                spec(ii).tmp_out.(fields{jj}) = any2str(spec(ii).tmp_out.(fields{jj}));
            catch ME
                true;
            end
        end

        out_nonsingle = [out_nonsingle;spec(ii).tmp_out_nonsingle];
        out = [out;spec(ii).tmp_out];
        gmt = [gmt,spec(ii).tmp_gmt];
        
        num_moas_represented = num_moas_represented + 1;
        
    else
        disp(moa_class)
        disp("NOT INCLUDED - NO NON-SINGLETON PCLS")
        
    end
    
end

num_moas_represented
numel(pcls)

num_moas_represented / numel(pcls)

1/111


In [52]:
gmt = cat(1,gmt{:});

In [59]:
sum([gmt.len]>2)
sum([gmt.len]<=2)

size(gmt, 1)


ans =

     4


ans =

     3


ans =

     7



In [40]:
% Save the outputs
wtable(out,fullfile(outdir,'pcls_spectral_clust.txt'))
wtable(out_nonsingle,fullfile(outdir,'pcls_spectral_clust_nonsingle.txt'))
mkgmt(fullfile(outdir,'pcls_spectral_clust.gmt'),gmt)